In [ ]:
"""This ipynb file help finds all the image that only have stroma"""

In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt

from config import Config
import utils
import model as modellib
import visualize
from model import log
import prostate

%matplotlib inline

# GPU to use
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
dataset_dir = "/data/wenyuan/Path_R_CNN/Data_Pre_Processing/cedars-224"
held_out_set = 4

In [ ]:
dataset_val = prostate.ProstateDataset()
_, val_list = dataset_val.generator_patition(dataset_dir, held_out_set)

In [ ]:
dataset_val.load_prostate(dataset_dir, val_list, mode = 16)
dataset_val.prepare()

In [ ]:
class InferenceConfig(prostate.ProstateConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0
    DETECTION_NMS_THRESHOLD = 1

inference_config = InferenceConfig()

In [ ]:
for image_id in range(0, len(dataset_val.image_ids), 16):
    stroma = 0 
    for patch in range(16):
        image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inference_config,
                               image_id + patch, use_mini_mask=False)
        tumor_class_gt = 1 \
        if (sum(gt_class_id)) else 0
        stroma += tumor_class_gt
    if (stroma == 0):
        print('Image_id : %d is all stroma' %val_list[int(image_id / 16)])